In [17]:
import pandas as pd
import numpy as np
import math

#### Read Data

In [18]:
df = pd.read_csv('D:\FirstSemMT\ML\Assignments\DLBCL.tab',sep='\t')
df.shape

(79, 7071)

In [19]:
df.head()

,A28102,AB000114_at,AB000115_at,AB000220_at,AB000409_at,AB000449_at,AB000450_at,AB000460_at,AB000462_at,AB000464_at,...,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at,class
0,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,DLBCL FL
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,class
2,-1,-45,176,97,-57,233,265,945,56,819,...,1036,-161,1523,-72,897,366,3,76,-28,DLBCL
3,25,-17,531,353,122,155,209,1688,42,639,...,4254,-75,6056,-68,737,81,-4,41,-45,DLBCL
4,73,91,257,80,614,507,760,2252,196,863,...,1934,-129,2901,-9,946,450,38,164,-62,DLBCL


In [20]:
df.drop(df.index[df['class'] == 'DLBCL FL'], inplace = True)
df.drop(df.index[df['class'] == 'class'], inplace = True)
label = df['class'] 
df.drop(['class'], axis = 1, inplace = True)
df = df.values
samples,features = df.shape
print(samples,features)

77 7070


In [21]:
data = df.astype(np.float)
data.shape

(77, 7070)

#### Import train_test_split function and Split dataset into training set and test set

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2,random_state=100) 

#### Find Mutual Info for every feature 

In [23]:
ig = np.zeros(features)

c1 = np.where(y_train=='DLBCL')[0]
c2 = np.where(y_train=='FL')[0]

u = np.array([c1.shape[0],c2.shape[0]])
ans1=0
ic = 0
for p in range(2):
    ans1 = ans1 - ((u[p]/u.sum())*math.log2(u[p]/(u.sum())))
ic = ans1            

for i in range(features): 
    minElement = np.amin(X_train[:,i])
    maxElement = np.amax(X_train[:,i])
    diff = (maxElement-minElement)/2
    
    a = minElement
    b = a+diff
    c = a+(2*diff)
    
    x = np.where(np.logical_and(X_train[:,i]>=a,X_train[:,i]<b))[0] 
    y = np.where(np.logical_and(X_train[:,i]>=b,X_train[:,i]<c))[0] 
    
    xc1=np.intersect1d(x,c1)
    yc1=np.intersect1d(y,c1)
    xc2=np.intersect1d(x,c2)
    yc2=np.intersect1d(y,c2)
    
    v = np.array([[xc1.shape[0],yc1.shape[0]],[xc2.shape[0],yc2.shape[0]]])
    ans2=0
    for p in range(2):
        for q in range(2):
            if(v[q][p]!=0):
                ans2 = ans2 - ((v[q][p]/v.sum())*math.log2(v[q][p]/(v[0][p]+v[1][p])))
    ig[i] = ic-ans2
print(ig)
    

[ 0.02932099 -0.00366207 -0.00656061 ...  0.00063523  0.01481685
  0.02739116]


#### Sort indexes in descending order of mutual info

In [24]:
sort_idxs=(-ig).argsort()[:ig.shape[0]]
sort_idxs

array([2040, 6119, 1313, ..., 4811, 2912, 2489], dtype=int64)

#### Select top features (less than 20%)

In [25]:
filt_idxs = sort_idxs[0:int((data.shape[1])*0.2)]

In [26]:
X_new_train = X_train[:,filt_idxs]
X_new_test = X_test[:,filt_idxs]
X_new_train.shape

(61, 1414)

#### KNN Classification

In [27]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_new_train, y_train)
y_pred = knn.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using KNN

In [28]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.6875
f1 score: 0.6761133603238866
confusion matrix: [[7 5]
 [0 4]]


#### SNM Classification

In [29]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_new_train, y_train)
y_pred = clf.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using SVM

In [30]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9375
f1 score: 0.9227053140096619
confusion matrix: [[11  1]
 [ 0  4]]
